In [1]:
from datasets import load_dataset

spanish_dataset = load_dataset("amazon_reviews_multi", "es")
english_dataset = load_dataset("amazon_reviews_multi", "en")
english_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to C:/Users/Raj/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [3]:
# let's sample the dataset
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}")
        print(f"\n'>> Review: {example['review_body']}")
        
show_samples(english_dataset)

Loading cached shuffled indices for dataset at C:\Users\Raj\.cache\huggingface\datasets\amazon_reviews_multi\en\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-0b126a62dc97ecba.arrow



'>> Title: Worked in front position, not rear

'>> Review: 3 stars because these are not rear brakes as stated in the item description. At least the mount adapter only worked on the front fork of the bike that I got it for.

'>> Title: meh

'>> Review: Does it’s job and it’s gorgeous but mine is falling apart, I had to basically put it together again with hot glue

'>> Title: Can't beat these for the money

'>> Review: Bought this for handling miscellaneous aircraft parts and hanger "stuff" that I needed to organize; it really fit the bill. The unit arrived quickly, was well packaged and arrived intact (always a good sign). There are five wall mounts-- three on the top and two on the bottom. I wanted to mount it on the wall, so all I had to do was to remove the top two layers of plastic drawers, as well as the bottom corner drawers, place it when I wanted and mark it; I then used some of the new plastic screw in wall anchors (the 50 pound variety) and it easily mounted to the wall. So

In [4]:
# training a summarization model on all 400k examples will take a long time on a single GPU.
# Let's focus on a single domain of products

english_dataset.set_format("pandas")
english_df = english_dataset["train"][:]
# show counts for top 20 product categories
english_df["product_category"].value_counts()[:20]

product_category
home                      17679
apparel                   15951
wireless                  15717
other                     13418
beauty                    12091
drugstore                 11730
kitchen                   10382
toy                        8745
sports                     8277
automotive                 7506
lawn_and_garden            7327
home_improvement           7136
pet_products               7082
digital_ebook_purchase     6749
pc                         6401
electronics                6186
office_product             5521
shoes                      5197
grocery                    4730
book                       3756
Name: count, dtype: int64

In [5]:
# let's filter the dataset to only include examples from the "Book" and digital "Ebook" categories from both languages
def filter_books(example):
    return example["product_category"] in ["book", "digital_ebook_purchase"]

In [ ]:
# apply the filter to the dataset
# before applying the filter, let's switch the format of english_dataset from "pandas" back to "arrow":
english_dataset.reset_format()

In [6]:
# apply the filter now
spanish_dataset = spanish_dataset.filter(filter_books)
english_dataset = english_dataset.filter(filter_books)
show_samples(english_dataset)

Filter:   0%|          | 0/200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]


'>> Title: 0    I'm dissapointed.
Name: review_title, dtype: object

'>> Review: 0    I guess I had higher expectations for this boo...
Name: review_body, dtype: object

'>> Title: 0    Good art, good price, poor design
Name: review_title, dtype: object

'>> Review: 0    I had gotten the DC Vintage calendar the past ...
Name: review_body, dtype: object

'>> Title: 0    Helpful
Name: review_title, dtype: object

'>> Review: 0    Nearly all the tips useful and. I consider mys...
Name: review_body, dtype: object
